<a href="https://colab.research.google.com/github/toluolatubosun/Simple-RAG-LLM-Chatbot/blob/main/RAG_LLM_Chatbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Install Packages
%pip install supabase
%pip install boto3
%pip install openai
%pip install tabulate

In [12]:
# Import the modules
import os
import json
import boto3
import supabase
from openai import OpenAI
from tabulate import tabulate
from google.colab import userdata

In [5]:
# Load Enviroment Variables

# Locally
# SUPABASE_URL = os.environ.get('SUPABASE_URL')
# SUPABASE_KEY = os.environ.get('SUPABASE_KEY')
# OPENAI_API_KEY = os.environ.get('OPENAI_API_KEY')

# Google Collab
SUPABASE_URL = userdata.get('SUPABASE_URL_2')
SUPABASE_KEY = userdata.get('SUPABASE_KEY_2')
OPENAI_API_KEY = userdata.get('OPENAI_API_KEY_2')

In [6]:
# Set Global Objects and Variables

# Supbase Client
supabase_client = supabase.create_client(SUPABASE_URL, SUPABASE_KEY)

# OpenAI
openai_client = OpenAI(api_key=OPENAI_API_KEY)

In [7]:
faq_data = [
    {
        "question": "What courses are offered at Mountain Top University?",
        "answer": "Mountain Top University offers programs in sciences, social sciences, humanities, management, and technology."
    },
    {
        "question": "Where is Mountain Top University located?",
        "answer": "It is located at KM 12, Lagos-Ibadan Expressway, Prayer City, Ogun State, Nigeria."
    },
    {
        "question": "Does Mountain Top University offer scholarships?",
        "answer": "Yes, the university offers merit-based and need-based scholarships to qualified students."
    },
    {
        "question": "What is the admission process for Mountain Top University?",
        "answer": "Admission is through UTME or direct entry. Candidates must meet departmental cut-off marks and pass the screening process."
    }
]

In [8]:
def get_embedding(text: str, model: str = "text-embedding-ada-002") -> list:
    response = openai_client.embeddings.create(
        input=text,
        model=model
    )
    return response.data[0].embedding

In [ ]:
def upload_faq_to_supabase():
    for item in faq_data:
        question = item["question"]
        answer = item["answer"]

        question_embedding = get_embedding(question)
        answer_embedding = get_embedding(answer)

        data = {
            "question": question,
            "answer": answer,
            "question_embedding": question_embedding,
            "answer_embedding": answer_embedding
        }

        supabase_client.table("faq_question").insert(data).execute()
        print(f"Uploaded: {question}")

upload_faq_to_supabase()



```sql
create or replace function match_faq_questions(
  input_embedding vector,
  similarity_threshold float,
  match_limit int
)
returns table (
  id uuid,
  question text,
  answer text,
  question_embedding vector,
  answer_embedding vector,
  similarity_score float
)
language sql
stable
as $$
  select
    fq.id,
    fq.question,
    fq.answer,
    fq.question_embedding,
    fq.answer_embedding,
    greatest(
      1 - (fq.question_embedding <=> input_embedding),
      1 - (fq.answer_embedding <=> input_embedding)
    ) as similarity_score
  from
    faq_questions fq
  where
    (1 - (fq.question_embedding <=> input_embedding)) >= similarity_threshold
    or (1 - (fq.answer_embedding <=> input_embedding)) >= similarity_threshold
  order by
    similarity_score desc
  limit match_limit;
$$;

```



In [ ]:
def search_faq(text: str, print_table: bool, threshold: float = 0.75, limit: int = 5):
    embedding = get_embedding(text)

    # Call Supabase RPC function
    response = supabase_client.rpc(
        "match_faq_questions",
        {
            "input_embedding": embedding,
            "similarity_threshold": threshold,
            "match_limit": limit
        }
    ).execute()

    if not response.data:
        print("No matches found.")
        return

    # Pretty print results
    rows = []
    for item in response.data:
        rows.append([
            round(item["similarity_score"], 3),
            item["question"],
            item["answer"]
        ])

    if print_table:
        print(tabulate(rows, headers=["Similarity", "Question", "Answer"], tablefmt="fancy_grid"))

    return response.data

search_faq("Location of MTU", True)
print("Maches gone")

In [ ]:
async def terminal_chatbot():
    print("💬 Welcome to the MTU FAQ Assistant. Type 'exit' to quit.\n")

    messages = [
        {
            "role": "system",
            "content": (
                "You are a helpful assistant for Mountain Top University. "
                "Use the provided tool to search the FAQ database when appropriate."
            )
        }
    ]

    # Define the tool
    faq_tool = {
        "type": "function",
        "function": {
            "name": "fetch_faq_tool",
            "description": "Searches MTU FAQ using a question",
            "parameters": {
                "type": "object",
                "properties": {
                    "text": {
                        "type": "string",
                        "description": "User question about Mountain Top University"
                    },
                },
                "required": ["text"]
            }
        }
    }

    while True:
        user_input = input("👤 You: ")
        if user_input.strip().lower() in ["exit", "quit"]:
            print("👋 Goodbye!")
            break

        messages.append({"role": "user", "content": user_input})

        # Call the API
        response = openai_client.chat.completions.create(
            model="gpt-4o",
            messages=messages,
            tools=[faq_tool],
            tool_choice="auto"
        )

        # Process the response
        response_message = response.choices[0].message
        tool_calls = response_message.tool_calls

        # Check if the model wants to call a tool
        if tool_calls:
            messages.append(response_message)  # extend conversation with assistant's reply

            for tool_call in tool_calls:
                if tool_call.function.name == "fetch_faq_tool":
                    args = json.loads(tool_call.function.arguments)
                    query_text = args["text"]

                    print(f"🛠️ Tool called: fetch_faq_tool → '{query_text}'")

                    result = search_faq(query_text, print_table=False)
                    answer = result[0]["answer"] if result else "No relevant FAQ found."

                    messages.append({
                        "tool_call_id": tool_call.id,
                        "role": "tool",
                        "name": tool_call.function.name,
                        "content": answer,
                    })

            # Get a new response from the model after providing tool output
            second_response = openai_client.chat.completions.create(
                model="gpt-4o",
                messages=messages,
            )
            assistant_reply = second_response.choices[0].message.content
            print(f"🤖 Assistant: {assistant_reply}\n")
            messages.append({"role": "assistant", "content": assistant_reply})

        else:
            # If no tool call, the response is the final answer
            assistant_reply = response_message.content
            print(f"🤖 Assistant: {assistant_reply}\n")
            messages.append({"role": "assistant", "content": assistant_reply})

await terminal_chatbot()